# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE
0,2020-01-01,EQ,1276.10,1280.00,1270.60,1272.10,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185
1,2020-01-02,EQ,1279.00,1288.00,1279.00,1278.60,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384
2,2020-01-03,EQ,1282.20,1285.00,1263.60,1286.75,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937
3,2020-01-06,EQ,1260.00,1261.80,1236.00,1268.40,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267
4,2020-01-07,EQ,1258.90,1271.45,1252.25,1240.95,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2024-02-20,EQ,1415.00,1457.25,1411.80,1417.10,1453.00,1454.30,1443.09,1757.5,1363.55,28066754,4.050282e+10,518597,17.641
1045,2024-02-21,EQ,1464.00,1464.00,1435.00,1454.30,1436.00,1439.15,1446.29,1757.5,1363.55,22946568,3.318731e+10,493970,17.457
1046,2024-02-22,EQ,1417.75,1428.80,1412.20,1439.15,1422.00,1419.55,1419.78,1757.5,1363.55,25280441,3.589274e+10,477326,17.219
1047,2024-02-23,EQ,1422.55,1433.90,1418.00,1419.55,1419.45,1420.60,1425.27,1757.5,1363.55,14916726,2.126042e+10,335851,17.232


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1049 non-null   datetime64[ns]
 1   Series      1049 non-null   object        
 2   Open        1049 non-null   float64       
 3   High        1049 non-null   float64       
 4   Low         1049 non-null   float64       
 5   Prev Close  1049 non-null   float64       
 6   LTP         1049 non-null   float64       
 7   Close       1049 non-null   float64       
 8   VWAP        1049 non-null   float64       
 9   52W H       1049 non-null   float64       
 10  52W L       1049 non-null   float64       
 11  Volume      1049 non-null   int64         
 12  Value       1049 non-null   float64       
 13  #Trades     1049 non-null   int64         
 14  PE          1049 non-null   float64       
dtypes: datetime64[ns](1), float64(11), int64(2), object(1)
memory usage: 175

## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh
545,2022-02-15,1476.00,1524.90,1473.05,1519.45,1517.80,1499.93,1725.0,1353.00,6567533,9.850826e+09,213639,26.287,51.85,1,0,0
329,2021-04-05,1480.00,1485.00,1431.00,1450.20,1449.60,1451.04,1641.0,826.10,8003293,1.161313e+10,238305,25.106,54.00,0,0,0
335,2021-04-13,1368.00,1406.45,1361.00,1401.00,1400.35,1383.23,1641.0,826.10,9300341,1.286454e+10,189809,24.253,45.45,1,0,0
1005,2023-12-26,1673.25,1685.95,1668.55,1684.10,1682.45,1678.46,1757.5,1460.25,9022928,1.514458e+10,243596,20.408,17.40,1,0,0
239,2020-11-20,1372.65,1408.40,1365.05,1405.00,1403.65,1391.31,1419.0,738.75,10202681,1.419512e+10,235627,28.242,43.35,1,0,0
763,2023-01-02,1627.00,1639.75,1618.55,1628.15,1628.70,1629.49,1722.1,1271.60,2190689,3.569697e+09,89456,23.735,21.20,1,0,0
354,2021-05-12,1399.75,1408.60,1388.85,1399.05,1399.50,1398.25,1641.0,826.10,6887463,9.630380e+09,189135,24.238,19.75,0,0,0
908,2023-08-03,1640.00,1651.35,1623.00,1632.60,1628.65,1633.08,1757.5,1365.00,28836973,4.709311e+10,439722,19.756,28.35,0,0,0
462,2021-10-18,1705.00,1725.00,1667.05,1674.50,1670.30,1690.00,1725.0,1176.35,14428228,2.438370e+10,413049,28.928,57.95,0,0,1
532,2022-01-27,1450.00,1485.00,1435.00,1482.00,1474.95,1454.91,1725.0,1342.00,14308919,2.081817e+10,398704,25.545,50.00,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      51.67
Is52WLow      1.72
Is52WHigh     2.48
dtype: float64

In [9]:
stock_df.shape

(1049, 17)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2024-02-20,1415.00,1457.25,1411.80,1453.00,1454.30,1443.09,1757.5,1363.55,28066754,4.050282e+10,518597,17.641,45.45,1,0,0,1
1045,2024-02-21,1464.00,1464.00,1435.00,1436.00,1439.15,1446.29,1757.5,1363.55,22946568,3.318731e+10,493970,17.457,29.00,0,0,0,1
1046,2024-02-22,1417.75,1428.80,1412.20,1422.00,1419.55,1419.78,1757.5,1363.55,25280441,3.589274e+10,477326,17.219,16.60,0,0,0,2
1047,2024-02-23,1422.55,1433.90,1418.00,1419.45,1420.60,1425.27,1757.5,1363.55,14916726,2.126042e+10,335851,17.232,15.90,1,0,0,1


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
31,2020-02-12,1245.20,1252.90,1244.05,1249.00,1249.00,1249.42,2503.30,1084.00,3148965,3.934383e+09,87413,30.463,8.85,1,0,0,2,12,2,2020,2,43,1,1
878,2023-06-21,1616.00,1637.25,1611.00,1635.55,1635.60,1628.22,1734.45,1294.15,12154473,1.979014e+10,276058,19.840,26.25,1,0,0,4,21,6,2023,2,172,2,1
918,2023-08-18,1587.55,1596.30,1585.50,1588.70,1590.75,1590.59,1757.50,1365.00,12816683,2.038613e+10,282239,19.296,10.80,0,0,0,6,18,8,2023,4,230,3,1
591,2022-04-26,1372.00,1381.95,1357.15,1370.45,1372.05,1371.84,1725.00,1292.00,8508421,1.167223e+10,295807,19.995,24.80,1,0,0,2,26,4,2022,1,116,2,1
327,2021-03-31,1548.00,1548.00,1488.00,1494.40,1493.65,1499.76,1641.00,810.00,15387253,2.307719e+10,362409,30.053,60.00,0,0,0,1,31,3,2021,2,90,1,1
451,2021-09-30,1586.00,1606.35,1583.10,1592.85,1594.95,1594.87,1641.00,1042.65,7987674,1.273928e+10,216914,27.623,23.25,1,0,0,1,30,9,2021,3,273,3,1
43,2020-02-20,1230.00,1230.00,1214.10,1217.00,1217.10,1221.38,2503.30,1084.00,5561707,6.792970e+09,146273,29.685,15.90,0,0,0,1,20,2,2020,3,51,1,1
755,2022-12-21,1641.00,1644.80,1606.55,1620.00,1617.60,1625.86,1722.10,1271.60,4634599,7.535207e+09,155853,23.573,38.25,0,0,0,2,21,12,2022,2,355,4,1
806,2023-03-03,1596.00,1620.00,1588.00,1615.25,1615.90,1610.57,1722.10,1271.60,7493117,1.206818e+10,147003,23.549,32.00,1,0,0,1,3,3,2023,4,62,1,1
860,2023-05-26,1619.00,1619.00,1602.25,1617.00,1615.80,1611.17,1734.45,1271.60,13317533,2.145685e+10,247378,19.600,16.75,1,0,0,1,26,5,2023,4,146,2,1


In [12]:
stock_df.shape

(1049, 25)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,1278.60,1278.60,1278.60,1278.60,1278.60
1,1286.75,1282.68,1282.68,1282.68,1282.68
2,1268.40,1277.92,1277.92,1277.92,1277.92
3,1240.95,1268.68,1268.68,1268.68,1268.68
4,1260.60,1267.06,1267.06,1267.06,1267.06
...,...,...,...,...,...
1044,1454.30,1410.55,1424.96,1483.78,1564.09
1045,1439.15,1417.57,1423.40,1476.73,1562.54
1046,1419.55,1421.16,1420.28,1468.85,1560.72
1047,1420.60,1426.38,1418.58,1461.24,1558.41


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,9.40,9.40,9.40,9.40,9.40
1,9.00,9.20,9.20,9.20,9.20
2,21.40,13.27,13.27,13.27,13.27
3,25.80,16.40,16.40,16.40,16.40
4,19.20,16.96,16.96,16.96,16.96
...,...,...,...,...,...
1044,45.45,26.84,27.34,31.17,26.69
1045,29.00,26.66,26.73,31.10,26.95
1046,16.60,26.17,26.67,31.03,27.03
1047,15.90,24.45,25.14,30.90,26.80


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,1276.64,1276.64,1276.64,1276.64,1276.64
1,1284.56,1280.60,1280.60,1280.60,1280.60
2,1270.48,1277.23,1277.23,1277.23,1277.23
3,1247.24,1269.73,1269.73,1269.73,1269.73
4,1261.48,1268.08,1268.08,1268.08,1268.08
...,...,...,...,...,...
1044,1443.09,1406.78,1424.99,1485.65,1564.58
1045,1446.29,1414.12,1423.93,1478.50,1563.20
1046,1419.78,1417.69,1420.89,1470.72,1561.34
1047,1425.27,1425.12,1418.45,1463.11,1559.30


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1836849,1836849,1836849,1836849,1836849
1,3068583,2452716,2452716,2452716,2452716
2,5427775,3444402,3444402,3444402,3444402
3,5445093,3944575,3944575,3944575,3944575
4,7362247,4628109,4628109,4628109,4628109
...,...,...,...,...,...
1044,28066754,24835397,25042707,29315026,23436816
1045,22946568,25652920,24380443,29657796,23682016
1046,25280441,25151484,24976115,30231881,23865945
1047,14916726,20759530,24474971,30423871,23830870


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,2.345001e+09,2345000988,2345000988,2345000988,2345000988
1,3.941792e+09,3143396262,3143396262,3143396262,3143396262
2,6.895886e+09,4394226092,4394226092,4394226092,4394226092
3,6.791348e+09,4993506527,4993506527,4993506527,4993506527
4,9.287302e+09,5852265530,5852265530,5852265530,5852265530
...,...,...,...,...,...
1044,4.050282e+10,34834049603,35607650809,43088402438,35953823802
1045,3.318731e+10,36142847916,34614834986,43493580958,36297099739
1046,3.589274e+10,35533571535,35410902474,44245958316,36531626007
1047,2.126042e+10,29613500480,34641987690,44449996588,36446970681


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,46625,46625,46625,46625,46625
1,104570,75597,75597,75597,75597
2,157066,102753,102753,102753,102753
3,155007,115817,115817,115817,115817
4,189026,130458,130458,130458,130458
...,...,...,...,...,...
1044,518597,509624,519356,634370,487788
1045,493970,510648,515719,641849,492829
1046,477326,515129,521198,651976,496542
1047,335851,436866,507114,655479,496889


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1240.95,1282.70,1240.85,1240.60
1,1260.60,1286.00,1244.85,1249.00
2,1257.30,1289.50,1244.55,1241.40
3,1271.40,1284.25,1213.20,1219.35
4,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...
1044,1420.60,NaN,NaN,NaN
1045,1422.30,NaN,NaN,NaN
1046,NaN,NaN,NaN,NaN
1047,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,9.40,9.40,9.40,9.40,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,9.20,9.20,9.20,9.20,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,13.27,13.27,13.27,13.27,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2,6,1,2020,0,6,1,3,1268.68,1268.68,1268.68,1268.68,16.40,16.40,16.40,16.40,1269.73,1269.73,1269.73,1269.73,3944575,3944575,3944575,3944575,4993506527,4993506527,4993506527,4993506527,115817,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1,7,1,2020,1,7,1,1,1267.06,1267.06,1267.06,1267.06,16.96,16.96,16.96,16.96,1268.08,1268.08,1268.08,1268.08,4628109,4628109,4628109,4628109,5852265530,5852265530,5852265530,5852265530,130458,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,2024-02-20,1415.00,1457.25,1411.80,1453.00,1454.30,1443.09,1757.5,1363.55,28066754,4.050282e+10,518597,17.641,45.45,1,0,0,1,20,2,2024,1,51,1,1,1410.55,1424.96,1483.78,1564.09,26.84,27.34,31.17,26.69,1406.78,1424.99,1485.65,1564.58,24835397,25042707,29315026,23436816,34834049603,35607650809,43088402438,35953823802,509624,519356,634370,487788,1420.60,NaN,NaN,NaN
1045,2024-02-21,1464.00,1464.00,1435.00,1436.00,1439.15,1446.29,1757.5,1363.55,22946568,3.318731e+10,493970,17.457,29.00,0,0,0,1,21,2,2024,2,52,1,1,1417.57,1423.40,1476.73,1562.54,26.66,26.73,31.10,26.95,1414.12,1423.93,1478.50,1563.20,25652920,24380443,29657796,23682016,36142847916,34614834986,43493580958,36297099739,510648,515719,641849,492829,1422.30,NaN,NaN,NaN
1046,2024-02-22,1417.75,1428.80,1412.20,1422.00,1419.55,1419.78,1757.5,1363.55,25280441,3.589274e+10,477326,17.219,16.60,0,0,0,2,22,2,2024,3,53,1,1,1421.16,1420.28,1468.85,1560.72,26.17,26.67,31.03,27.03,1417.69,1420.89,1470.72,1561.34,25151484,24976115,30231881,23865945,35533571535,35410902474,44245958316,36531626007,515129,521198,651976,496542,NaN,NaN,NaN,NaN
1047,2024-02-23,1422.55,1433.90,1418.00,1419.45,1420.60,1425.27,1757.5,1363.55,14916726,2.126042e+10,335851,17.232,15.90,1,0,0,1,23,2,2024,4,54,1,1,1426.38,1418.58,1461.24,1558.41,24.45,25.14,30.90,26.80,1425.12,1418.45,1463.11,1559.30,20759530,24474971,30423871,23830870,29613500480,34641987690,44449996588,36446970681,436866,507114,655479,496889,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,1.958,0.848,1.0,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,0.998,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,1.945,0.842,1.0,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.007,0.998,0.995,0.995,0.995,0.995,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.974,0.855,1.0,1.008,1.008,1.008,1.008,0.017,0.010,0.010,0.010,0.010,1.002,1.007,1.007,1.007,1.007,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,2.017,0.874,1.0,1.022,1.022,1.022,1.022,0.021,0.013,0.013,0.013,0.013,1.005,1.023,1.023,1.023,1.023,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.986,0.860,1.0,1.005,1.005,1.005,1.005,0.015,0.013,0.013,0.013,0.013,1.001,1.006,1.006,1.006,1.006,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0.973,1.002,0.971,0.999,1.208,0.938,1.0,0.970,0.980,1.020,1.075,0.031,0.018,0.019,0.021,0.018,0.992,0.967,0.980,1.022,1.076,0.977,NaN,NaN,NaN
1045,1.017,1.017,0.997,0.998,1.221,0.947,1.0,0.985,0.989,1.026,1.086,0.020,0.019,0.019,0.022,0.019,1.005,0.983,0.989,1.027,1.086,0.988,NaN,NaN,NaN
1046,0.999,1.007,0.995,1.002,1.238,0.961,1.0,1.001,1.001,1.035,1.099,0.012,0.018,0.019,0.022,0.019,1.000,0.999,1.001,1.036,1.100,NaN,NaN,NaN,NaN
1047,1.001,1.009,0.998,0.999,1.237,0.960,1.0,1.004,0.999,1.029,1.097,0.011,0.017,0.018,0.022,0.019,1.003,1.003,0.998,1.030,1.098,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.799,0.799,0.799,0.799
2,1.0,0.635,0.635,0.635,0.635
3,1.0,0.724,0.724,0.724,0.724
4,1.0,0.629,0.629,0.629,0.629
...,...,...,...,...,...
1044,1.0,0.885,0.892,1.044,0.835
1045,1.0,1.118,1.062,1.292,1.032
1046,1.0,0.995,0.988,1.196,0.944
1047,1.0,1.392,1.641,2.040,1.598


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.797,0.797,0.797,0.797
2,1.0,0.637,0.637,0.637,0.637
3,1.0,0.735,0.735,0.735,0.735
4,1.0,0.630,0.630,0.630,0.630
...,...,...,...,...,...
1044,1.0,0.860,0.879,1.064,0.888
1045,1.0,1.089,1.043,1.311,1.094
1046,1.0,0.990,0.987,1.233,1.018
1047,1.0,1.393,1.629,2.091,1.714


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.723,0.723,0.723,0.723
2,1.0,0.654,0.654,0.654,0.654
3,1.0,0.747,0.747,0.747,0.747
4,1.0,0.690,0.690,0.690,0.690
...,...,...,...,...,...
1044,1.0,0.983,1.001,1.223,0.941
1045,1.0,1.034,1.044,1.299,0.998
1046,1.0,1.079,1.092,1.366,1.040
1047,1.0,1.301,1.510,1.952,1.479


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0.973,1.002,0.971,0.999,0.992,1.208,0.938,17.641,1,0,0,1,20,2,2024,1,51,1,1,0.970,0.980,1.020,1.075,0.018,0.019,0.021,0.018,0.967,0.980,1.022,1.076,0.885,0.892,1.044,0.835,0.860,0.879,1.064,0.888,0.983,1.001,1.223,0.941,0.977,NaN,NaN,NaN
1045,1.017,1.017,0.997,0.998,1.005,1.221,0.947,17.457,0,0,0,1,21,2,2024,2,52,1,1,0.985,0.989,1.026,1.086,0.019,0.019,0.022,0.019,0.983,0.989,1.027,1.086,1.118,1.062,1.292,1.032,1.089,1.043,1.311,1.094,1.034,1.044,1.299,0.998,0.988,NaN,NaN,NaN
1046,0.999,1.007,0.995,1.002,1.000,1.238,0.961,17.219,0,0,0,2,22,2,2024,3,53,1,1,1.001,1.001,1.035,1.099,0.018,0.019,0.022,0.019,0.999,1.001,1.036,1.100,0.995,0.988,1.196,0.944,0.990,0.987,1.233,1.018,1.079,1.092,1.366,1.040,NaN,NaN,NaN,NaN
1047,1.001,1.009,0.998,0.999,1.003,1.237,0.960,17.232,1,0,0,1,23,2,2024,4,54,1,1,1.004,0.999,1.029,1.097,0.017,0.018,0.022,0.019,1.003,0.998,1.030,1.098,1.392,1.641,2.040,1.598,1.393,1.629,2.091,1.714,1.301,1.510,1.952,1.479,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0.973,1.002,0.971,0.999,0.992,1.208,0.938,17.641,1,0,0,1,20,2,2024,1,51,1,1,0.970,0.980,1.020,1.075,0.018,0.019,0.021,0.018,0.967,0.980,1.022,1.076,0.885,0.892,1.044,0.835,0.860,0.879,1.064,0.888,0.983,1.001,1.223,0.941,0.977,NaN,NaN,NaN
1045,1.017,1.017,0.997,0.998,1.005,1.221,0.947,17.457,0,0,0,1,21,2,2024,2,52,1,1,0.985,0.989,1.026,1.086,0.019,0.019,0.022,0.019,0.983,0.989,1.027,1.086,1.118,1.062,1.292,1.032,1.089,1.043,1.311,1.094,1.034,1.044,1.299,0.998,0.988,NaN,NaN,NaN
1046,0.999,1.007,0.995,1.002,1.000,1.238,0.961,17.219,0,0,0,2,22,2,2024,3,53,1,1,1.001,1.001,1.035,1.099,0.018,0.019,0.022,0.019,0.999,1.001,1.036,1.100,0.995,0.988,1.196,0.944,0.990,0.987,1.233,1.018,1.079,1.092,1.366,1.040,NaN,NaN,NaN,NaN
1047,1.001,1.009,0.998,0.999,1.003,1.237,0.960,17.232,1,0,0,1,23,2,2024,4,54,1,1,1.004,0.999,1.029,1.097,0.017,0.018,0.022,0.019,1.003,0.998,1.030,1.098,1.392,1.641,2.040,1.598,1.393,1.629,2.091,1.714,1.301,1.510,1.952,1.479,NaN,NaN,NaN,NaN
